In [1]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

In [2]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [31]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic
sig, sr = torchaudio.load("file.wav")
print(sig.shape)
Audio(data=sig, rate=sr)

torch.Size([1, 125448])


In [7]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, AutoFeatureExtractor,pipeline

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [22]:
!pip install librosa

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00


In [20]:
from datasets import load_dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [32]:
pipe(sig[0].numpy())

{'text': ' Yes, sir.',
 'chunks': [{'timestamp': (0.0, 3.84), 'text': ' Yes, sir.'}]}

In [34]:
pipe("file.wav")


{'text': " What's for dinner today?",
 'chunks': [{'timestamp': (0.0, 2.62), 'text': " What's for dinner today?"}]}

In [38]:
prompt = "Translate the audio into hindi"
prompt_ids = processor.get_prompt_ids(prompt)

In [39]:
prompt_ids

array([50362,  6531, 17593,   264,  6278,   666,   276, 10255])

In [42]:
def convert_forced_to_tokens(forced_decoder_ids):
    # decode tokens if not None
    forced_decoder_tokens = []
    for i, (idx, token) in enumerate(forced_decoder_ids):
        if token is not None:
            forced_decoder_tokens.append([idx, processor.tokenizer.decode(token)])
        else:
            forced_decoder_tokens.append([idx, token])
    return forced_decoder_tokens

In [49]:
idx = processor.tokenizer.all_special_tokens.index("<|startofprev|>")
forced_bos_token_id = processor.tokenizer.all_special_ids[105]

prompt = " translate to hindi"
prompt_tokens = processor.tokenizer(prompt, add_special_tokens=False).input_ids

# we need to force these tokens
forced_decoder_ids = []
for idx, token in enumerate(prompt_tokens):
    # indexing starts from 1 for forced tokens (token at position 0 is the SOS token)
    forced_decoder_ids.append([idx + 1, token])
    
# now we add the SOS token at the end
offset = len(forced_decoder_ids)
forced_decoder_ids.append([offset + 1, model.generation_config.decoder_start_token_id])

# now we need to append the rest of the prefix tokens (lang, task, timestamps)
offset = len(forced_decoder_ids)
for idx, token in model.generation_config.forced_decoder_ids:
    forced_decoder_ids.append([idx + offset + 1, token])

In [50]:
forced_decoder_tokens = convert_forced_to_tokens(forced_decoder_ids)
print(forced_decoder_tokens)

[[1, ' translate'], [2, ' to'], [3, ' h'], [4, 'indi'], [5, '<|startoftranscript|>'], [7, '<|en|>'], [8, '<|transcribe|>'], [9, '<|notimestamps|>']]


In [51]:
model.config.forced_decoder_ids = None
model.generation_config.forced_decoder_ids = None
model.generation_config.is_multilingual = False


In [47]:
model.generation_config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="English", task="transcribe")

forced_decoder_ids = model.generation_config.forced_decoder_ids
print(forced_decoder_ids)

forced_decoder_tokens = convert_forced_to_tokens(forced_decoder_ids)
print(forced_decoder_tokens)

[(1, 50259), (2, 50360), (3, 50364)]
[[1, '<|en|>'], [2, '<|transcribe|>'], [3, '<|notimestamps|>']]


In [57]:
from librosa import resample

In [58]:
res = resample(sig[0].numpy(),orig_sr=sr,target_sr=16_000)

In [59]:
res

array([ 1.2151496e-03,  1.7396300e-03,  1.3029951e-03, ...,
        1.2784109e-03, -1.4129677e-05,  2.7908338e-04], dtype=float32)

In [60]:
#input_features = processor(sample["audio"]["array"], sampling_rate=sample["audio"]["sampling_rate"], return_tensors="pt").input_features
input_features = processor(res, sampling_rate=16_000, return_tensors="pt").input_features




In [68]:
input_features = input_features.half()

In [65]:
input_features

tensor([[[-0.1199, -0.2524,  0.0338,  ..., -0.5939, -0.5939, -0.5939],
         [-0.0224, -0.1548,  0.1314,  ..., -0.5939, -0.5939, -0.5939],
         [-0.2477,  0.1516,  0.2806,  ..., -0.5939, -0.5939, -0.5939],
         ...,
         [-0.5939, -0.5814, -0.5784,  ..., -0.5939, -0.5939, -0.5939],
         [-0.5697, -0.5765, -0.5586,  ..., -0.5939, -0.5939, -0.5939],
         [-0.5939, -0.5939, -0.5939,  ..., -0.5939, -0.5939, -0.5939]]],
       device='cuda:0')

In [69]:
pred_ids = model.generate(input_features, max_new_tokens=128, decoder_start_token_id=forced_bos_token_id, forced_decoder_ids=forced_decoder_ids)
output = processor.batch_decode(pred_ids, skip_special_tokens=False)
print(output)

["<|startofprev|> translate to hindi<|startoftranscript|><|en|><|en|><|transcribe|><|notimestamps|> what's for dinner today<|endoftext|>"]
